In [1]:
import pandas as pd
import requests
import json
from pprint import pprint
from sqlalchemy import create_engine

In [2]:
# divvy stations feed url
url = "https://feeds.divvybikes.com/stations/stations.json"

In [6]:
response = requests.get(url).json()
# pprint(response)

In [7]:
stations = pd.DataFrame(response['stationBeanList'])

In [10]:
stations.head()

,altitude,availableBikes,availableDocks,city,id,is_renting,kioskType,landMark,lastCommunicationTime,latitude,...,longitude,postalCode,stAddress1,stAddress2,stationName,status,statusKey,statusValue,testStation,totalDocks
0,,30,8,Chicago,2,True,K3_FULL,15541,2019-06-12 10:53:30,41.876470,...,-87.620340,60605,Buckingham Fountain,,Buckingham Fountain,IN_SERVICE,1,In Service,False,39
1,,16,39,Chicago,3,True,K3_FULL,15544,2019-06-12 10:52:53,41.867226,...,-87.615355,,Shedd Aquarium,,Shedd Aquarium,IN_SERVICE,1,In Service,False,55
2,,21,2,Chicago,4,True,K3_FULL,15545,2019-06-12 10:56:43,41.856268,...,-87.613348,,Burnham Harbor,,Burnham Harbor,IN_SERVICE,1,In Service,False,23
3,,2,20,Chicago,5,True,K3_FULL,13030,2019-06-12 10:55:33,41.874053,...,-87.627716,60605,State St & Harrison St,,State St & Harrison St,IN_SERVICE,1,In Service,False,23
4,,14,22,Chicago,6,True,K3_FULL,15548,2019-06-12 10:55:19,41.886976,...,-87.612813,60601,Dusable Harbor,,Dusable Harbor,IN_SERVICE,1,In Service,False,39


In [12]:
stations["lastCommunicationTime"] = pd.to_datetime(stations["lastCommunicationTime"])

In [13]:
stations.dtypes

altitude                         object
availableBikes                    int64
availableDocks                    int64
city                             object
id                                int64
is_renting                         bool
kioskType                        object
landMark                         object
lastCommunicationTime    datetime64[ns]
latitude                        float64
location                         object
longitude                       float64
postalCode                       object
stAddress1                       object
stAddress2                       object
stationName                      object
status                           object
statusKey                         int64
statusValue                      object
testStation                        bool
totalDocks                        int64
dtype: object

In [ ]:
database = {'user': 'divvy', 
            'password': '1234', 
            'port': '3306',
            'host': 'localhost',
            'database': 'divvy_db' }

db_engine = create_engine("""mysql://%s:%s@%s:%s/%s
    """ % (database["user"], database["password"], database["host"], database["port"], database["database"]),
    echo=False)

stations.to_sql("stations", con=db_engine, if_exists="replace", chunksize=20000)